In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.autograd import Variable
import torch.backends.cudnn as cudnn

from torchvision.utils import save_image

import numpy as np

import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms, models, datasets

In [6]:
import glob

In [7]:
os.path.exists("./output/000")

False

In [8]:
for epoch in range (200):
  if (os.path.exists("./output/%03d" % epoch)) == False:
    os.mkdir("./output/%03d" % epoch)
  else:
    files = glob.glob("./output/%03d/*.png" % epoch)

    for f in files:
      os.remove(f)

# Data

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

kwargs = {'num_workers': 2, 'pin_memory': True}

cuda = True
image_size = 32
batchSize = 16

In [10]:
def show_img(source, target, source_label, target_label):
    num_row = 4
    num_col = 5
    num = 10
    images = source[:num]
    labels = source_label[:num]

    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
    for i in range(num):
        ax = axes[i//num_col, i%num_col]
        
        image =  images[i].transpose(0,2).transpose(0,1)

        ax.imshow(image, cmap='gray')
        ax.set_title('Label: {}'.format(labels[i]))


    images = target[:num]
    labels = target_label[:num]
    for i in range(10,20):
        ax = axes[i//num_col, i%num_col]
        image = images[i - 10].transpose(0,2).transpose(0,1)
        ax.imshow(image, cmap='gray')
        ax.set_title('Label: {}'.format(labels[i - 10]))
    plt.tight_layout()
    plt.show()

## Mnist

In [11]:
transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Lambda(lambda t: t * 2 - 1)])

mnist_trainset = datasets.MNIST(root='./data/mnist', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data/mnist', train=False, download=True, transform=transform)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



## Mnist-M

In [12]:
def get_backgrounds():
    backgrounds = []
    for file in os.listdir("./images/train"):
        if file.endswith('.jpg'):
            backgrounds.append(plt.imread(os.path.join("./images/train",file)))
    return backgrounds

def compose_image(image, backgrounds):
    image = (image > 0).astype(np.float32)
    image = image.reshape([28,28])*255.0
    
    image = np.stack([image,image,image],axis=2)
    
    background = np.random.choice(backgrounds)
    w,h,_ = background.shape
    dw, dh,_ = image.shape
    x = np.random.randint(0,w-dw)
    y = np.random.randint(0,h-dh)
    
    temp = background[x:x+dw, y:y+dh]
    return np.abs(temp-image).astype(np.uint8)

class MNISTM(Dataset):    
    def __init__(self, train=True,transform=None):
        if train:
            self.data = datasets.MNIST(root='.data/mnist',train=True, download=True)
        else:
            self.data = datasets.MNIST(root='.data/mnist',train=False, download=True)
        self.backgrounds = get_backgrounds()
        self.transform = transform
        self.images = []
        self.targets = []
        for index in range(len(self.data)):
            image = np.array(self.data.__getitem__(index)[0])
            target = self.data.__getitem__(index)[1]
            image = compose_image(image, self.backgrounds)
            if self.transform is not None:
                image = self.transform(image)
            self.images.append(image)
            self.targets.append(target)
        
    def __getitem__(self,index):
        image = self.images[index]
        target = self.targets[index]
        
        return image, target
        
    def __len__(self):
        return len(self.data)

In [13]:
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Lambda(lambda t: t * 2 - 1)
        ])

trainset = MNISTM(train=True,transform=transform)
testset = MNISTM(train=False,transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to .data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting .data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to .data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [14]:
source_train = DataLoader(mnist_trainset, batch_size=batchSize, shuffle=True, drop_last=True, **kwargs)
source_test = DataLoader(mnist_testset, batch_size=batchSize, shuffle=True, drop_last=True, **kwargs)

In [15]:
target_train = DataLoader(trainset, batch_size=batchSize, shuffle=True, drop_last=True, **kwargs)
target_test = DataLoader(testset, batch_size=batchSize, shuffle=False, drop_last=True, **kwargs)

In [16]:
source_iter = iter(source_train)
source_inputs, source_label = source_iter.next()

target_iter = iter(target_train)
target_inputs, target_label = target_iter.next()

In [17]:
print("%2f, %2f" % (torch.min(source_inputs), torch.max(source_inputs)))

-1.000000, 1.000000


In [18]:
print("%2f, %2f" % (torch.min(target_inputs), torch.max(target_inputs)))

-1.000000, 1.000000


# Module

## Perceptual

In [19]:
class Perceptual(nn.Module):
    def __init__(self):
        super(Perceptual, self).__init__()

        self.encoder = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7),
            nn.InstanceNorm2d(3),
            nn.Tanh()
        )

    def forward(self, A, B):
        encode_A = self.encoder(A)
        encode_B = self.encoder(B)
        reconA = self.decoder(encode_A)
        reconB = self.decoder(encode_B)

        encode_A.detach()
        encode_B.detach()

        style = encode_A[:, 0:128, : , :]
        content = encode_B[:, 128:256, :, :]
        
        mixed_latent = torch.cat([style, content], dim=1)
        mixed_image = self.decoder(mixed_latent)

        return mixed_image, reconA, reconB

In [20]:
test_tensor_source = source_inputs
test_tensor_target = target_inputs

perceptual = Perceptual()
encoder = perceptual.encoder
decoder = perceptual.decoder
encoder_out = encoder(test_tensor_target)
decoder_out = decoder(encoder_out)

print(encoder_out.size())
print(decoder_out.size())

mixed, reconA, reconB = perceptual(test_tensor_source, test_tensor_target)
print(mixed.size())
print(reconA.size())
print(reconB.size())

print("mixed: min: %.2f, max: %.2f " % (torch.min(mixed), torch.max(mixed)))

print("reconA: min: %.2f, max: %.2f " % (torch.min(reconA), torch.max(reconA)))

print("reconB: min: %.2f, max: %.2f " % (torch.min(reconB), torch.max(reconB)))

torch.Size([16, 256, 8, 8])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
mixed: min: -1.00, max: 1.00 
reconA: min: -1.00, max: 1.00 
reconB: min: -1.00, max: 1.00 


In [21]:
print(mixed[0][0][0] - reconA[0][0][0])
print(mixed[0][0][0] - reconB[0][0][0])

tensor([-0.0024,  0.5574, -0.2707,  0.3759,  0.1369,  0.1579, -0.1070,  0.4516,
        -0.0817,  0.2587,  0.1346,  0.0104,  0.2323,  0.0125,  0.5082,  0.4452,
         0.2734, -0.3078,  0.3854, -0.0409,  0.1057,  0.1208, -0.4163,  0.0565,
         0.8344,  0.2597, -0.0312, -0.6774,  0.1288, -0.1791, -0.4352,  0.0521],
       grad_fn=<SubBackward0>)
tensor([-0.2501,  0.5314, -0.4970,  0.0590, -0.1905,  0.1932, -0.3552, -0.3277,
        -0.2159,  0.7255, -0.3727, -0.1092,  0.0641,  0.4412, -0.3247, -0.5027,
         0.4121,  0.6994,  0.2032, -0.0766, -0.4103,  0.1763, -0.0301, -0.0493,
         0.8141,  0.7995,  0.0771,  0.2380,  0.1582,  0.1779,  0.2316, -0.0674],
       grad_fn=<SubBackward0>)


## Discriminator

In [22]:
class Discriminator(nn.Module):
    def __init__(self, input_nc = 3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, padding=1),
            nn.InstanceNorm2d(128), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 1, 4, padding=1)
        )


    def forward(self, x):
        x =  self.model(x)
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

In [23]:
discriminator = Discriminator()

output = discriminator(test_tensor_source)
print(output.size())
print(torch.min(output))
print(torch.max(output))

torch.Size([16, 1])
tensor(-0.2590, grad_fn=<MinBackward1>)
tensor(-0.2033, grad_fn=<MaxBackward1>)


# Loss

In [24]:
def tv_loss(img, tv_weight=5e-2):
    w_variance = torch.sum(torch.pow(img[:,:,:,:-1] - img[:,:,:,1:], 2))
    h_variance = torch.sum(torch.pow(img[:,:,:-1,:] - img[:,:,1:,:], 2))
    loss = tv_weight * (h_variance + w_variance)
    return loss

def total_variation_loss(img, weight=5e-2):
    bs_img, c_img, h_img, w_img = img.size()
    tv_h = torch.pow(img[:, :, 1:, :] - img[:, :, :-1, :], 2).sum()
    tv_w = torch.pow(img[:, :, :, 1:] - img[:, :, :, :-1], 2).sum()
    return weight * (tv_h + tv_w) / (bs_img * c_img * h_img * w_img)

def compute_content_loss(target_feature, content_feature):
    return torch.mean((target_feature - content_feature)**2)

def batch_gram_matrix(img):
    b, d, h, w = img.size()
    img = img.view(b*d, h*w) # fix the dimension. It doesn't make sense to put b=1 when it's not always the case
    gram = torch.mm(img, img.t())
    return gram

style_weights = {'conv1_1': 1.0, 'conv2_1': 0.75, 'conv3_1': 0.2, 'conv4_1': 0.2, 'conv5_1': 0.2}

def compute_style_loss(style_features, target_features):
    style_loss = 0
    style_grams = {layer: batch_gram_matrix(style_features[layer]) for layer in style_features}

    for layer in style_weights:
        # get the "target" style representation for the layer
        target_feature = target_features[layer]
        target_gram = batch_gram_matrix(target_feature)
        _, d, h, w = target_feature.shape

        style_gram = style_grams[layer]

        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
        style_loss += layer_style_loss / (d * h * w)

    return style_loss

# Training

## Training Method

In [25]:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '19': 'conv4_1', '21': 'conv4_2',  '28': 'conv5_1'}

    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features

## Model hypermeter

In [26]:
perceptual = Perceptual()
discriminator = Discriminator()

In [27]:
learning_rate = 0.002
beta = (0.5, 0.999)

criterion_adv = torch.nn.MSELoss()
criterion_discriminator = torch.nn.MSELoss()
criterion_construct = torch.nn.L1Loss()

optimizer_pre = torch.optim.Adam(perceptual.parameters(), lr=learning_rate, betas=beta)
optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=beta)

In [28]:
vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
    param.requires_grad_(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [29]:
if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True

    criterion_construct = criterion_construct.cuda()
    criterion_discriminator = criterion_discriminator.cuda()
    criterion_construct = criterion_construct.cuda()

    perceptual.cuda()
    discriminator.cuda()

## Init the network

In [30]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [31]:
perceptual.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

## Training

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

real_label = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
fake_label = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)
real_label = real_label[:, None]
fake_label = fake_label[:, None]


input_A = Tensor(batchSize, 3, image_size, image_size)
input_B = Tensor(batchSize, 3, image_size, image_size)

for epoch in range(0, 200):
    i = -1
    for batchA, batchB in zip(target_train, source_train):
        i+= 1
        real_A = Variable(input_A.copy_(batchA[0]))
        real_B = Variable(input_B.copy_(batchB[0]))

        optimizer_pre.zero_grad()
        mixed_image, reconstructionA, reconstructionB = perceptual(real_A, real_B)

        loss_ss = criterion_construct(reconstructionA, real_A) * 30.0
        loss_tt = criterion_construct(reconstructionB, real_B) * 30.0

        TV_loss = total_variation_loss(mixed_image)
        pred_fake = discriminator(mixed_image)
        loss_adv = criterion_adv(pred_fake, real_label)

        cuda_mixed_image = mixed_image.clone().requires_grad_(True).cuda()
        cuda_real_A = real_A.clone().requires_grad_(True).cuda()
        cuda_real_B = real_B.clone().requires_grad_(True).cuda()
        style_features = get_features(cuda_real_A, vgg)
        content_features = get_features(cuda_real_B, vgg)
        target_features = get_features(cuda_mixed_image, vgg)

        content_loss = compute_content_loss(target_features['conv4_2'], content_features['conv4_2']) * 0.1
        style_loss = compute_style_loss(style_features, target_features) * 0.05

        preceptual_loss = loss_ss + loss_tt + TV_loss + loss_adv + content_loss + style_loss

        preceptual_loss.backward()
        optimizer_pre.step()

        optimizer_dis.zero_grad()

        pred_real = discriminator(real_A)
        pred_fake = discriminator(mixed_image.detach())

        loss_dis_real = criterion_discriminator(pred_real, real_label)
        loss_dis_fake = criterion_discriminator(pred_real, fake_label)

        discriminator_loss = loss_dis_real + loss_dis_fake
        discriminator_loss.backward()
        optimizer_dis.step()

        if  i % 200 == 0:
            real_A = real_A.data
            real_B = real_B.data
            mixed_image = mixed_image.data
            reconstructionA = reconstructionA.data
            reconstructionB = reconstructionB.data

            save_image(real_A, './output/%03d/00_%d_A.png' % ( epoch, i))
            save_image(real_B, 'output/%03d/00_%d_B.png' % ( epoch, i))
            save_image(reconstructionA, 'output/%03d/00_%d_reconA.png' % ( epoch, i))
            save_image(reconstructionB, 'output/%03d/00_%d_reconB.png' % ( epoch, i))
            save_image(mixed_image, 'output/%03d/00_%d_Mixed.png' % ( epoch, i))
        
    print ("e: %d, pre_loss: %.2f, D_real: %.2f, D_fake: %.2f" % (epoch, preceptual_loss, loss_dis_real, loss_dis_fake))
    

e: 0, pre_loss: 23.08, D_real: 0.27, D_fake: 0.23
e: 1, pre_loss: 22.55, D_real: 0.27, D_fake: 0.23
e: 2, pre_loss: 22.12, D_real: 0.25, D_fake: 0.25
e: 3, pre_loss: 21.13, D_real: 0.32, D_fake: 0.19
e: 4, pre_loss: 21.28, D_real: 0.33, D_fake: 0.18
e: 5, pre_loss: 21.80, D_real: 0.23, D_fake: 0.27
e: 6, pre_loss: 20.98, D_real: 0.15, D_fake: 0.37
e: 7, pre_loss: 22.34, D_real: 0.25, D_fake: 0.25
e: 8, pre_loss: 22.26, D_real: 0.33, D_fake: 0.18
e: 9, pre_loss: 20.25, D_real: 0.25, D_fake: 0.25
e: 10, pre_loss: 22.33, D_real: 0.25, D_fake: 0.25
e: 11, pre_loss: 19.97, D_real: 0.25, D_fake: 0.25
e: 12, pre_loss: 22.22, D_real: 0.25, D_fake: 0.25
e: 13, pre_loss: 21.52, D_real: 0.26, D_fake: 0.24
e: 14, pre_loss: 21.35, D_real: 0.26, D_fake: 0.25
e: 15, pre_loss: 20.53, D_real: 0.28, D_fake: 0.22
e: 16, pre_loss: 22.74, D_real: 0.24, D_fake: 0.26
e: 17, pre_loss: 21.04, D_real: 0.25, D_fake: 0.26
e: 18, pre_loss: 20.88, D_real: 0.27, D_fake: 0.23
e: 19, pre_loss: 21.63, D_real: 0.25, D_f

In [ ]:
torch.save(perceptual, "./model_weight/perceptual")
torch.save(discriminator, "./model_weight/discriminator")

In [ ]:
from PIL import Image

from pylab import imread,subplot,imshow,show

import matplotlib.pyplot as plt

image = imread('./output/199/00_400_reconA.png')

plt.imshow(image)



In [ ]:
image = imread('./output/199/00_400_Mixed.png')

plt.imshow(image)

In [ ]:
image = imread('./output/199/00_400_reconB.png')

plt.imshow(image)